In [1]:
# Import dependencies
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [2]:
# Read in data
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


# Select features (columns)

In [4]:
# Set X equal to the entire data set, except for the first column
X = df.iloc[:,1:]

# Set y equal to the first column
y = df.iloc[:,0]

In [5]:
# Search for top 10 features according to feature importances
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

array([0.13220615, 0.16999607, 0.1346137 , 0.04534865, 0.01813975,
       0.01108751, 0.01582232, 0.01057706, 0.02321712, 0.02318696,
       0.01589067, 0.0117528 , 0.00905199, 0.01562369, 0.02898498,
       0.02657171, 0.0165266 , 0.01136696, 0.01071499, 0.01891792,
       0.01258985, 0.01561753, 0.01426325, 0.0092176 , 0.00800232,
       0.00878718, 0.03307985, 0.00779219, 0.01113198, 0.01651553,
       0.01563764, 0.0113813 , 0.0104469 , 0.01539599, 0.00879613,
       0.01000379, 0.00998183, 0.01126239, 0.01035272, 0.01014644])

In [6]:
# Store the top 10 features as a series, using the column headers as the index
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_ss        0.169996
koi_fpflag_co        0.134614
koi_fpflag_nt        0.132206
koi_fpflag_ec        0.045349
koi_model_snr        0.033080
koi_duration_err1    0.028985
koi_duration_err2    0.026572
koi_time0bk_err1     0.023217
koi_time0bk_err2     0.023187
koi_prad             0.018918
dtype: float64

In [7]:
# Set features based on feature importances
X = df[feat_imp.index]

# Use koi_disposition for y values
y = df['koi_disposition']

# Create a Train Test Split

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler

In [9]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler object
X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model
Using SVM


In [10]:
from sklearn.svm import SVC

# Initialize model
model = SVC(kernel='linear')

# Train the model
model.fit(X_train_scaled, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [11]:
# Score the model
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8113675376692733
Testing Data Score: 0.8112128146453089


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [12]:
from sklearn.model_selection import GridSearchCV

# Create the GridSearchCV model
param_grid = [{'C':[1, 5, 10, 50], 'kernel':['linear']},
             {'C':[1, 5, 10, 50], 'kernel':['rbf'], 'gamma': [0.0001, 0.0005, 0.001, 0.005]}]

grid = GridSearchCV(model, param_grid, verbose=3)

In [13]:
# Train the model with GridSearch
best_model = grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.797, total=   0.1s
[CV] C=1, kernel=linear ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] .................. C=1, kernel=linear, score=0.797, total=   0.1s
[CV] C=1, kernel=linear ..............................................
[CV] .................. C=1, kernel=linear, score=0.797, total=   0.1s
[CV] C=5, kernel=linear ..............................................
[CV] .................. C=5, kernel=linear, score=0.840, total=   0.1s
[CV] C=5, kernel=linear ..............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] .................. C=5, kernel=linear, score=0.834, total=   0.1s
[CV] C=5, kernel=linear ..............................................
[CV] .................. C=5, kernel=linear, score=0.835, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.848, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.844, total=   0.1s
[CV] C=10, kernel=linear .............................................
[CV] ................. C=10, kernel=linear, score=0.844, total=   0.1s
[CV] C=50, kernel=linear .............................................
[CV] ................. C=50, kernel=linear, score=0.852, total=   0.1s
[CV] C=50, kernel=linear .............................................
[CV] ................. C=50, kernel=linear, score=0.855, total=   0.1s
[CV] C=50, kernel=linear .............................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   20.6s finished


In [16]:
print(best_model.best_params_)
print("Best Score: ", best_model.best_score_)

{'C': 50, 'kernel': 'linear'}
Best Score:  0.8531375166889186


In [15]:
grid_predictions = best_model.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":grid_predictions}) 
df_grid.head()

,Actual,Predicted
1569,CONFIRMED,CONFIRMED
2371,FALSE POSITIVE,FALSE POSITIVE
3130,CANDIDATE,CONFIRMED
3101,FALSE POSITIVE,FALSE POSITIVE
414,CONFIRMED,CONFIRMED


# Predict With GridSearch Parameters

In [17]:
# Initialize model (classifier)
model = SVC(kernel='linear', C=50)

# Train the model
model.fit(X_train_scaled, y_train)

SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [18]:
# Make predictions on test data
df2 = pd.DataFrame({'Actual':y_test, 'Predicted':model.predict(X_test_scaled)})
df2.head()

,Actual,Predicted
1569,CONFIRMED,CONFIRMED
2371,FALSE POSITIVE,FALSE POSITIVE
3130,CANDIDATE,CONFIRMED
3101,FALSE POSITIVE,FALSE POSITIVE
414,CONFIRMED,CONFIRMED


In [19]:
print(model.score(X_test_scaled, y_test))

0.8609839816933639
